In [2]:
from preprocessing import Audio
import tensorflow as tf
import tf_train as tft
import numpy as np
import os
import matplotlib.pyplot as plt
import scipy as sp
import librosa as li
import librosa.display
import IPython.display as ipd

%matplotlib inline
#tf.debugging.set_log_device_placement(True)

In [3]:
GPU = tf.config.list_logical_devices('GPU')
CPU = tf.config.list_logical_devices('CPU')
DEVICE = GPU[0].name if GPU else CPU[0].name
DEVICE

'/device:GPU:0'

<h1>Подготовка данных для обучения и тестирования</h1>

In [86]:
def get_data():
    def norm(x):
        std = np.std(x ,ddof=1)
        return (x - np.mean(x)) / std

    PATH_0 = r'AudioData/0 Данные' # negative class
    PATH_1 = r'AudioData/1 Итан' # positive class

    PATHS = li.util.find_files(PATH_0)[:10000] + li.util.find_files(PATH_1)
    N = len(PATHS)

    np.random.shuffle(PATHS)

    SOUNDS = []
    for path in PATHS:
        SOUNDS.append(Audio(path))
        print('Loading {:.1f}%'.format(len(SOUNDS) / N * 100))
        ipd.clear_output(wait=True)
    DATA = []
    LABELS = []

    for sound in SOUNDS:
        DATA.append(norm(sound.e_parts))
        LABELS.append(sound.label)

    return (np.array(DATA), np.array(LABELS))

In [87]:
data, labels = get_data()

Loading 100.0%


In [88]:
test_data, test_labels = data[8000:], labels[8000:]
data, labels = data[:8000], labels[:8000]

In [91]:
data.shape, labels.shape

((8000, 20), (8000,))

In [92]:
dataset = tf.data.Dataset.from_tensor_slices((data, labels))
dataset = dataset.batch(1000)
dataset

<BatchDataset shapes: ((None, 20), (None,)), types: (tf.float64, tf.int32)>

In [188]:
data = np.reshape(data, (10070, 20))

In [211]:
labels = np.reshape(labels, (10070,))

In [209]:
labels

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])

<h1>Создание модели нейронной сети</h1>

In [98]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(12, activation='tanh', kernel_regularizer=tf.keras.regularizers.l2(0.01),
                          input_shape=(20,)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2, activation='softmax')
])

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 12)                252       
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 26        
Total params: 278
Trainable params: 278
Non-trainable params: 0
_________________________________________________________________


In [204]:
y_pred = model.predict(data)
print(y_pred[:10])
y_pred = np.argmax(y_pred, axis=-1)
print(y_pred[:10])
print(labels[:10])

[[9.9888867e-01 1.1113590e-03]
 [9.9939823e-01 6.0171861e-04]
 [9.7778863e-01 2.2211321e-02]
 [9.9991632e-01 8.3627907e-05]
 [9.5554715e-01 4.4452846e-02]
 [9.9310726e-01 6.8927575e-03]
 [9.9917573e-01 8.2422997e-04]
 [9.9672639e-01 3.2735907e-03]
 [9.9872667e-01 1.2733137e-03]
 [9.9947780e-01 5.2216806e-04]]
[0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0]


# Клонирование модели для transfer learning

In [9]:
model = tf.keras.models.load_model('Джарвис.h5')
model = tf.keras.models.clone_model(model)
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 12)                252       
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 26        
Total params: 278
Trainable params: 278
Non-trainable params: 0
_________________________________________________________________


<h1>Обучение</h1>

In [100]:
tft.train(
    train_dataset=dataset,
    model=model,
    epochs=1000,
    loss=tf.keras.losses.SparseCategoricalCrossentropy()
)

Epoch 000 -- Loss(SparseCategoricalCrossentropy): 0.010, Recall: 0.6020088195800781, Precision: 0.8771690726280212
Epoch 010 -- Loss(SparseCategoricalCrossentropy): 0.010, Recall: 0.6022405624389648, Precision: 0.8772915005683899
Epoch 020 -- Loss(SparseCategoricalCrossentropy): 0.009, Recall: 0.6025061011314392, Precision: 0.8774194717407227
Epoch 030 -- Loss(SparseCategoricalCrossentropy): 0.009, Recall: 0.6027705669403076, Precision: 0.8775469064712524
Epoch 040 -- Loss(SparseCategoricalCrossentropy): 0.009, Recall: 0.6030722856521606, Precision: 0.8777099847793579
Epoch 050 -- Loss(SparseCategoricalCrossentropy): 0.009, Recall: 0.6033728122711182, Precision: 0.8778918981552124
Epoch 060 -- Loss(SparseCategoricalCrossentropy): 0.009, Recall: 0.6036720275878906, Precision: 0.87807297706604
Epoch 070 -- Loss(SparseCategoricalCrossentropy): 0.009, Recall: 0.6039699912071228, Precision: 0.878253161907196
Epoch 080 -- Loss(SparseCategoricalCrossentropy): 0.009, Recall: 0.6042667031288147

In [18]:
model.save('Mira-v1.0.h5')

In [101]:
predictions = tft.predict_classes(model=model, features=test_data)

In [102]:
n = 0

for i in range(len(test_labels)):
    n += int(test_labels[i] == predictions[i])

print(n, n / len(labels) * 100)

2046 25.575


In [31]:
predictions[:10], labels[:10]

(<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 1, 0, 0, 0, 1, 0, 0, 0, 0], dtype=int64)>,
 array([0, 1, 0, 0, 0, 1, 0, 0, 0, 0]))

In [39]:
model.predict(data[:2])

array([[9.9999857e-01, 1.4108425e-06],
       [3.2748736e-05, 9.9996722e-01]], dtype=float32)

In [40]:
labels[:2]

array([0, 1])

In [69]:
filename = 'test.tfrecord'
writer_data = tf.data.experimental.TFRecordWriter("datasets\Mira\data.tfrecord")
writer_labels = tf.data.experimental.TFRecordWriter("datasets\Mira\labels.tfrecord")
test_data = tf.data.Dataset.from_tensor_slices(data)
test_labels = tf.data.Dataset.from_tensor_slices(labels)
test_data = test_data.map(lambda x: str(x))
test_labels = test_labels.map(lambda x: str(x))
writer_data.write(test_data)
writer_labels.write(test_labels)

In [75]:
loaded_dataset = tf.data.TFRecordDataset(["datasets\Mira\labels.tfrecord"])

In [76]:
for d in loaded_dataset.take(1):
    print(d)

tf.Tensor(b'Tensor("args_0:0", shape=(), dtype=int32)', shape=(), dtype=string)


In [77]:
loaded_dataset

<TFRecordDatasetV2 shapes: (), types: tf.string>

In [83]:
for d in test_data.take(1):
    print(d.numpy())

b'Tensor("args_0:0", shape=(20,), dtype=float64)'


In [84]:
data[0]

array([-1.67636868, -1.63167534, -1.07627448, -0.92766935, -0.09969852,
       -0.79751682, -0.47027632,  1.5961253 ,  1.63171389,  1.30207031,
        0.55290537,  1.14207755, -0.64697944, -0.13645845, -0.15388176,
        0.74642466,  0.9525158 ,  0.10332707, -0.43498627,  0.02462547])